In [105]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag ,pos_tag_sents
import matplotlib.pyplot as plt
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import vstack
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.model_selection import StratifiedKFold

In [2]:
embeddings_dict = {}

In [3]:
stop_words=set(stopwords.words("english"))

In [4]:
with open('glove.twitter.27B.50d.txt', 'r' ,encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        emb =np.asarray(values[1:], "float32")
        embeddings_dict[word] = emb

In [57]:
'is' in embeddings_dict.keys()

True

In [6]:

def remove_link(content):
    '''remove links (using map) and hashtags and mentions'''
    a = content[:content.find('http' or 'https')]
    a = re.sub('#\w+', '', a)
    a = re.sub('@\w+','',a)
    return a

In [7]:
class LemmaTokenizer(object):
    '''lemmatize,tokenize,lower'''
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.token = RegexpTokenizer(r'[a-zA-Z]+')
    def __call__(self, articles):
        return [self.wnl.lemmatize(t).lower() for t in self.token.tokenize(articles) if t.lower() not in stopwords.words('english')]

In [8]:
data = pd.read_csv('SemEval2017-task4-dev.subtask-A.english.INPUT.txt', names = ['id', 'label', 'content'], sep ='\t', index_col=False)
data.head()

,id,label,content
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195045888,negative,If these runway renovations at the airport pre...
3,619974445185302528,neutral,If you could ask an onstage interview question...
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...


In [9]:
data.content = data.content.map(remove_link)

In [10]:
# tokenize each content

In [11]:
lt = LemmaTokenizer()
data['token'] = data.content.map(lt)

In [12]:
data.head()

,id,label,content,token
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","[picturehouse, pink, floyd, roger, waters, wal..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,"[order, go, set, watchman, store, website, tue..."
2,619971047195045888,negative,If these runway renovations at the airport pre...,"[runway, renovation, airport, prevent, seeing,..."
3,619974445185302528,neutral,If you could ask an onstage interview question...,"[could, ask, onstage, interview, question, mis..."
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,"[portion, book, sale, harper, lee, go, set, wa..."


In [49]:
def embedder(x):
    '''embed each token with glove (if it exists in DS), also make the vector size 20'''
    t =[]
    for y in x:
        try:
            t.append(embeddings_dict[y])
        except:
            pass
    while len(t)<20:
        t.append([0]*50)
    
    return np.array(t[:20])

In [50]:
#find number of unique words count


In [14]:
word_index= []
for token in data.token:
    for t in token:
        word_index.append(t)

In [36]:
len(word_index)

224552

In [16]:
word_index_len = len(list(set(word_index)))
word_index_len

20665

In [51]:
data['vector']=data.token.map(embedder)

In [60]:
data.head()

,id,label,content,token,vector,integer_token
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","[picturehouse, pink, floyd, roger, waters, wal...","[[0.16812999546527863, 0.2084600031375885, 0.3...","[9982, 1578, 241, 2725, 7033, 776, 334, 483, 1..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,"[order, go, set, watchman, store, website, tue...","[[-0.5772899985313416, -0.24309000372886658, -...","[670, 18, 81, 360, 894, 1721, 72, 14, 417, 109..."
2,619971047195045888,negative,If these runway renovations at the airport pre...,"[runway, renovation, airport, prevent, seeing,...","[[-1.4539999961853027, -0.14899000525474548, 0...","[5574, 9984, 2364, 4701, 312, 432, 671, 19, 97..."
3,619974445185302528,neutral,If you could ask an onstage interview question...,"[could, ask, onstage, interview, question, mis...","[[0.01691400073468685, 0.480679988861084, -0.3...","[179, 512, 7034, 672, 498, 156, 293, 2, 47, -1..."
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,"[portion, book, sale, harper, lee, go, set, wa...","[[-0.07840699702501297, -0.4659999907016754, 0...","[5575, 313, 613, 967, 808, 18, 81, 360, 433, 3..."


In [39]:
#convert tokens to integer indexes
counts = Counter(word_index)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word_index: ii for ii, word_index in enumerate(vocab, 1)}

In [69]:
def token2integer(token):
    x= [vocab_to_int[t] for t in token if t in embeddings_dict.keys() ]
    while len(x)<20:
        x.append(0)
    return x[:20]

In [70]:
data['integer_token'] = data.token.map(token2integer)

In [71]:
X = []
for i in data.integer_token:
    X.append(i)

In [72]:
X= np.array(X)
X.shape

(20632, 20)

In [73]:
len(vocab_to_int)

20665

## embedding matrix: in contains embedding for each word

In [74]:
embedding_matrix = np.zeros([len(vocab_to_int)+1 , 50])
for word, i in vocab_to_int.items():
    try:
        embedding_matrix[i] = embeddings_dict[word]
    except:
        pass

In [75]:
embedding_matrix.shape

(20666, 50)

In [76]:
#test train split and embedding labeld

In [77]:
le = LabelEncoder()
data['encoded_label'] = le.fit_transform(data.label)

In [141]:
batch_size = 128


In [593]:
# now added LSTM

In [108]:

from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input
acc_per_fold = []
loss_per_fold = []
fold_num = 1
kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=7)
for train, test in kfold.split(X, data.encoded_label.values):
    model=Sequential()
    model.add(Embedding(word_index_len + 1,
                                50,
                                weights=[embedding_matrix],
                                input_length=20,
                                trainable=False))
    model.add(LSTM(128)) # model.add(LSTM(64,return_sequences=True))

    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'],learning_rate= 0.01)
    num_epochs = 6

    model.fit(X[train],data.encoded_label.values[train], batch_size=batch_size, epochs=num_epochs,)
    scores = model.evaluate(X[test],data.encoded_label.values[test], verbose=0)
    print('fold',fold_num, 'acc:',scores[1])
    fold_num += 1
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])
#print(model.summary())

Train on 17683 samples
Epoch 1/6
17683/17683 [==============================] - 14s 786us/sample - loss: 0.8754 - accuracy: 0.5783
Epoch 2/6
17683/17683 [==============================] - 10s 574us/sample - loss: 0.7951 - accuracy: 0.6293
Epoch 3/6
17683/17683 [==============================] - 10s 554us/sample - loss: 0.7737 - accuracy: 0.6459
Epoch 4/6
17683/17683 [==============================] - 10s 541us/sample - loss: 0.7527 - accuracy: 0.6552
Epoch 5/6
17683/17683 [==============================] - 10s 542us/sample - loss: 0.7382 - accuracy: 0.6628
Epoch 6/6
17683/17683 [==============================] - 10s 560us/sample - loss: 0.7268 - accuracy: 0.6730
fold 1 acc: 0.65615463
Train on 17683 samples
Epoch 1/6
17683/17683 [==============================] - 14s 779us/sample - loss: 0.8710 - accuracy: 0.5842
Epoch 2/6
17683/17683 [==============================] - 10s 548us/sample - loss: 0.8005 - accuracy: 0.6288
Epoch 3/6
17683/17683 [==============================] - 10s 576us/

In [109]:
acc_per_fold

[0.65615463, 0.66327566, 0.6439471, 0.6470987, 0.6687033, 0.6602172, 0.6731161]

In [110]:
np.mean(acc_per_fold)

0.6589304

In [111]:
np.mean(loss_per_fold)

0.748691299476224

In [138]:
x_predict = np.zeros([20,50])
for t in range(X[0].shape[0]):
    x_predict[t] = embedding_matrix[X[0,t]]

In [148]:
model.predict_proba(x_predict.reshape([50,20]))

InvalidArgumentError:  indices[0,9] = -1 is not in [0, 20666)
	 [[node sequential_24/embedding_22/embedding_lookup (defined at E:\anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_148281]

Function call stack:
distributed_function


In [139]:
x_predict.shape

(20, 50)

In [121]:
X[0].shape

(20,)

In [143]:
model.input_shape

(None, 20)

In [465]:
#word2vec with gensim

In [466]:
import gensim 
from gensim.models import Word2Vec 

In [481]:
main_token = []

In [482]:
for t in data.token:
    main_token.append(t.)

In [484]:
type(main_token)

list

In [568]:
model2 = gensim.models.Word2Vec(main_token, min_count = 1, size = 100, window = 10, sg = 1) 

In [570]:
def w2v_embedder(x):
    '''embed each token with glove (if it exists in DS), also make the vector size 20'''
    t =[]
    for y in x:
        try:
            t.append(model2[y])
        except:
            pass
    while len(t)<20:
        t.append([0]*100)
    
    return np.array(t[:20])

In [571]:
data['vector']=data.token.map(w2v_embedder)

C:\Users\98914\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [572]:
data.iloc[0].vector.shape

(20, 100)

In [633]:
vocab_to_int['floyd']

241